#1 Gerekli Kütüphaneleri import edelim

In [ ]:
import numpy as np # lineer Cebir
import pandas as pd # cvs dosyalarını okumak için
import matplotlib.pyplot as plt
import os,fnmatch
import librosa
import IPython.display as ipd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
d_yolu=["/content/gdrive/MyDrive/My/data/"]

##2 Bölüm Mel-FC Göre öznitelik çıkarma

In [ ]:
# DataFrame sutunlarını belirleyelim.
sutunlar=["mfkk"+str(i) for i in range(20)]
for isim in ["zero","centroid","rolloff","chroma","class","dosya"]:
    sutunlar.append(isim)

In [ ]:
classes_liste=["Ali*.wav","Arzu*.wav","Dilek*.wav","elif*.wav","Reyyan*.wav","Zeliha*.wav"]

In [ ]:
# burdaki maksat verilen ses dosyaları mel-frekans özelliklerine göre hesaplamamk
def ses_df(d_yolu,sutunlar,classes_liste):
    liste=[]
    adim=0
    for folder in d_yolu:
        for classes in classes_liste:
            dosyalar=fnmatch.filter(os.listdir(folder),classes)
            label=classes.split("*")[0]
            for dosya in dosyalar:
                x,sr=librosa.load(folder+dosya,duration=5,res_type='kaiser_fast') # kayıt kalitesi yüksek olsun
                liste.append([np.mean(x) for x in librosa.feature.mfcc(x,sr=sr)])
                #mel frekans özelliklerine göre tek tek hesaplıyoruz
                liste[adim].append(sum(librosa.zero_crossings(x)))
                liste[adim].append(np.mean(librosa.feature.spectral_centroid(x)))
                liste[adim].append(np.mean(librosa.feature.spectral_rolloff(x,sr=sr)))
                liste[adim].append(np.mean(librosa.feature.chroma_stft(x,sr=sr)))
                liste[adim].append(label)
                liste[adim].append(dosya)
                adim+=1
    return pd.DataFrame(liste,columns=sutunlar)

In [ ]:
sound_df=ses_df(d_yolu,sutunlar,classes_liste)  # veri seti paketleri okunur

3 dakikaya yakın okuma süresi

300 kayıt okundu ve 26 öznitelik çıkartıldı

In [ ]:
sound_df.shape # veri seti boyutu

In [ ]:
sound_df.head(10) # baştan verilen sayı kadar eleman göster

In [ ]:
sound_df["class"].value_counts() # sınıflardaki verilerin ne kadar olduğunu görüyoruz

In [ ]:
X=sound_df.iloc[:,0:5] # o aralıktaki satır boyu kadar olacak kısımlara kadar olan bilgiyi görüntüle
y=sound_df["class"]
y

0         Ali
1         Ali
2         Ali
3         Ali
4         Ali
        ...  
295    Zeliha
296    Zeliha
297    Zeliha
298    Zeliha
299    Zeliha
Name: class, Length: 300, dtype: object

#**3- Eğitim ve Test Kümelerini Oluşturma.**

In [ ]:
random_state = np.random.RandomState(31) #rastgele verilerin gelmesi için   (0 ile 2**32 - 1 )

In [ ]:
#Train ve Test olmak üzere oluşturduğumuz öznitelik verisini bölüyoruz.
from sklearn.model_selection import train_test_split
X_train,,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=random_state)
print("X Train: ",len(X_train),"\n","X Test: ",len(X_test),sep="")

X Train: 240
X Test: 60
